In [27]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import zipfile
import os
import requests

In [45]:
# Step 1: Ensure Dataset Directory Exists
train_dir = "C:\\Users\\deepi\\Downloads\\archive (2)\\train"
test_dir = "C:\\Users\\deepi\\Downloads\\archive (2)\\test"

if not os.path.exists(train_dir) or not os.path.exists(test_dir):
    raise FileNotFoundError("Dataset not found. Please download the FER2013 dataset from Kaggle, extract it, and set the correct paths.")


In [46]:
# Step 2: Data Augmentation and Preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [47]:
train_data = data_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_data = ImageDataGenerator(rescale=1.0/255.0).flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')


Found 11431 images belonging to 4 classes.
Found 3867 images belonging to 4 classes.


In [48]:
# Handle class mismatch by aligning test data to training classes
if train_data.num_classes != test_data.num_classes:
    print("Mismatch in number of classes between train and test datasets. Adjusting test data to match training classes.")
    class_indices = train_data.class_indices
    test_data.class_indices = {k: v for k, v in test_data.class_indices.items() if k in class_indices}
    test_data.num_classes = len(test_data.class_indices)


In [49]:
# Step 3: Load Pre-trained Model and Fine-Tune
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [50]:
# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

In [51]:
# Add custom layers
gap = GlobalAveragePooling2D()(base_model.output)
dropout = Dropout(0.5)(gap)
output_layer = Dense(train_data.num_classes, activation='softmax')(dropout)

In [52]:
model = Model(inputs=base_model.input, outputs=output_layer)


In [53]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [54]:
# Step 4: Train the Model
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10,
    steps_per_epoch=train_data.samples // train_data.batch_size,
    validation_steps=test_data.samples // test_data.batch_size)


Epoch 1/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 596s 2s/step - accuracy: 0.3419 - loss: 1.3767 - val_accuracy: 0.2714 - val_loss: 1.2530
Epoch 2/10
  1/357 ━━━━━━━━━━━━━━━━━━━━ 7:03 1s/step - accuracy: 0.3125 - loss: 1.2874

C:\Users\deepi\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3125 - loss: 1.2874 - val_accuracy: 0.1481 - val_loss: 1.3945
Epoch 3/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 560s 2s/step - accuracy: 0.3362 - loss: 1.2865 - val_accuracy: 0.2669 - val_loss: 1.2492
Epoch 4/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3125 - loss: 1.1956 - val_accuracy: 0.2222 - val_loss: 1.2787
Epoch 5/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 547s 2s/step - accuracy: 0.3333 - loss: 1.2590 - val_accuracy: 0.2497 - val_loss: 1.2665
Epoch 6/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3438 - loss: 1.3359 - val_accuracy: 0.2963 - val_loss: 1.2846
Epoch 7/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 609s 2s/step - accuracy: 0.3520 - loss: 1.2340 - val_accuracy: 0.2555 - val_loss: 1.2524
Epoch 8/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4062 - loss: 1.3695 - val_accuracy: 0.2593 - val_loss: 1.2729
Epoch 9/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 560s 2s/step - accuracy: 0.3529 - loss: 1.2286 - val_accuracy: 0.2643 - 

In [55]:
# Step 5: Fine-Tuning (Unfreezing Layers for Better Performance)
for layer in base_model.layers:
    layer.trainable = True

In [57]:
# Recompile with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])


In [58]:
# Continue Training
history_finetune = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10,
    steps_per_epoch=train_data.samples // train_data.batch_size,
    validation_steps=test_data.samples // test_data.batch_size)

Epoch 1/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 2073s 6s/step - accuracy: 0.3705 - loss: 1.9774 - val_accuracy: 0.2641 - val_loss: 23.2320
Epoch 2/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.4062 - loss: 1.2051 - val_accuracy: 0.4074 - val_loss: 18.8560
Epoch 3/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 2041s 6s/step - accuracy: 0.4844 - loss: 1.1663 - val_accuracy: 0.2674 - val_loss: 2.4369
Epoch 4/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.4688 - loss: 1.1177 - val_accuracy: 0.0000e+00 - val_loss: 2.7589
Epoch 5/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 2024s 6s/step - accuracy: 0.5461 - loss: 1.0226 - val_accuracy: 0.5247 - val_loss: 1.0796
Epoch 6/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.5000 - loss: 0.9535 - val_accuracy: 0.5556 - val_loss: 1.0753
Epoch 7/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 2033s 6s/step - accuracy: 0.5967 - loss: 0.9391 - val_accuracy: 0.6263 - val_loss: 0.9203
Epoch 8/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.5312 - loss: 1.0255 - v

KeyboardInterrupt: 

In [59]:
# Step 6: Save the Model
model.save("emotion_recognition_resnet50.h5")


In [60]:
# Step 7: Evaluate the Model
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

121/121 ━━━━━━━━━━━━━━━━━━━━ 163s 1s/step - accuracy: 0.6360 - loss: 0.9161
Test Loss: 0.9301362037658691, Test Accuracy: 0.6242565512657166
